## Importing libraries

In the next cell we're importing all the needed libraries to manage and process MIDI files.


In [ ]:
from mydy import Events, FileIO, Containers, Constants

# Automatic MIDI extraction from multiple files

## Quantizing following Bars

The following cell contains a support function in order to quantize and "snap" bars. <br>
Since our code translates a whole dataset of MIDI files into a single .txt file, it will append sequentially all the MIDI files. <br>
This function ensures that sequential MIDI files are appended correctly.

In [ ]:
#function to approximate te length to the next bar https://stackoverflow.com/questions/3407012/c-rounding-up-to-the-nearest-multiple-of-a-number
def roundup(numToRound, multiple):
    if multiple == 0:
        return numToRound
    remainder = numToRound % multiple
    if remainder==0:
        return numToRound
    return numToRound + multiple - remainder

    

## Reading notes

I'm setting the  ```resolution ``` parameter to 480. This parameter is equivalent to **PPQ** in MIDI files.

**PPQ** (*Pulse per Quarter Note*) is a fixed value which sets the number of pulses contained in a quarter note, it's like a "sampling" frequency.

Each quarter note (no matter what is the original speed of the song) will contain 480 pulses. Then these pulses are converted into actual playback using the **Tempo** information of the MIDI file (obviously a quarter note at 100 BPM is slower than a quarter note at 130 BPM)

The following cell access the loaded MIDI file and reads the  ```Track```. Using the  ```mydy``` library, MIDI files are structured in the following way:

 ```Pattern -> Track -> MIDI_Events ```
 
 This means that, whenever I open a MIDI file, i will get a  ```Pattern ```, which contains  ```Track ```, which contains ```MIDI_Events ```.
 
 This scructure is used because a single MIDI file can contain multiple instruments (guitar, drums, bass ecc...), so each  ```Track ``` corresponds to an instrument, and the  ```MIDI_Events ``` are the note played by the single instrument.
 
 In our case we can assume to be working with single  ```Track ``` MIDI files (we're interested only in drums).
 
 **NB**: i'm using  ```track.make_ticks_abs()``` to convert the time (expressed in ticks/PPQ) from a relative value into an absolute one.<br>The standard representation of MIDI files represents note based on the time elapsed by the previous note.
With this command i'm converting the time from relative to absolute (each note is represented with the time elapsed by the beginning of the song) 

The following function basically iterates of the Dataset folder, reads the MIDI files and appends the many ```Track``` objects in a single ```Pattern``` object.

In [ ]:
import os
#Fix here the problem with the tick. Now i'm just adding the last tick to the next track. This makes the new track starting right from there
#instead of waiting for the new BAR. I have to quantize it and make it start from the new BAR 
pattern = Containers.Pattern(fmt=0)
max_time=0;
approx_bar = 480*4
pattern.resolution= 480
for filename in os.listdir("Dataset NEW"):
    print(os.path.join("Dataset NEW",filename))
    path= os.path.join("Dataset NEW",filename)
    
    test=FileIO.read_midifile(path)
    test.resolution = 480
    test=test[0].make_ticks_abs()
    test=test.filter(lambda e: isinstance(e, Events.NoteOnEvent))# Selects only Note_On events, i'm discarding the note off
    print("printo la singola track")
    print(test)
    for note in test: #probabile bug qui! il valore massimo di tick con i dati attuali dovrebbe essere 61320! BUG QUI! SICURO!
        note.tick = note.tick + max_time
    print('max time before')
    print(max_time)
    max_time = test[-1].tick
    #insert here round up on max_time
    max_time=roundup(max_time, approx_bar)
    print('max time after')
    print(max_time)
    
    pattern.append(test)
    
print(pattern)



The following cell simply extracts the tracks and appends it into a list. We need to get rid of the ```Pattern``` object, we're only interested in the ```Track``` objects. 

In [ ]:
filtered_list = []

for track in pattern:
    print("printo una track")
    print(track)
    filtered_list.append(track)
len(filtered_list)

## Creating couples (Pitch, time)
So basically now i'm processing the previous raw data in order to obtain couples of ```(pitch, tick)``` for each note. ```Tick``` is basically the timing of a note with respect to the beginning of the MIDI file. <br>

We're basically saying: "*This note is played after n ticks*"


In [ ]:
notes = []
couple = ()
for element in filtered_list:
    for note in element:
        ### test printings
        print(note.tick) #
        print(note.data[0]) #note pitch
    
    
        notes.append(tuple((note.data[0],note.tick))) #storing couples of (note_pitch,tick_time)
        



In [ ]:
#RANDOM TEST ON THE LIBRARY, USELESS
test= Constants.C_3
print(test)

## Trouble and make it double
Now we're going into the tough part.

The following cell is an *utility* in order to convert our MIDI into a text file (that we will use to traing our network, read the reference model here: https://arxiv.org/pdf/1604.05358v1.pdf)

Basically it creates 2 classes: ```Note``` and ```Note_List```.

**1)** *Note*:    Creates a simple ```Note``` object composed by ```pitch``` ```c_tick``` and ```idx```. ```pitch``` and ```c_tick``` are the already mentioned pitch and time, while ```idx``` is a variable that counts the index of my note on a 16-th note reference. <br>**For example**: if i have 2 whole notes, the second note will have ```idx=16```.

**2)** *Note_list*:  Creates an empty list where we will add our *Note* objects. It also contains supports methods used to create and manage this list.



In [ ]:
from mydy import Events, FileIO, Containers
import pdb

#Function responsible for converting midi notes into text. Since i have to train my network over the structure i decided
#which is 0b0000000 for no note, 0b01000000 for kick ecc... i need to convert midi notes into this format.

#The original script used for midi-text translation has been lost, must be re-implemented again
PPQ = 480 # Pulse per quater note. Used in sequencers. Standard value
event_per_bar = 16 # to quantise.
min_ppq = PPQ / (event_per_bar/4)

# ignore: 39 hand clap, 54 tambourine, 56 Cowbell, 58 Vibraslap, 60-81

#the dictionary below maps values to other ones. Reduced the size of the used notes. For example
#if i have an eletric snare or a stick snare, i just map both of them into a standard snare

drum_conversion = {35:36, # acoustic bass drum -> bass drum (36)
                    37:38, 40:38, # 37:side stick, 38: acou snare, 40: electric snare
                    43:41, # 41 low floor tom, 43 ghigh floor tom
                    47:45, # 45 low tom, 47 low-mid tom
                    50:48, # 50 high tom, 48 hi mid tom
                    44:42, # 42 closed HH, 44 pedal HH
                    57:49, # 57 Crash 2, 49 Crash 1
                    59:51, 53:51, 55:51, # 59 Ride 2, 51 Ride 1, 53 Ride bell, 55 Splash
                    52:49 # 52: China cymbal
                    }

#Used in the code to map elements, everything that has not one of the following number is discarded.
#Basically i'm ignoring notes that are not in my dataset (for examle i'll ignore shakers ecc...)
                # k, sn,cHH,oHH,LFtom,ltm,htm,Rde,Crash
allowed_pitch = [36, 38, 42, 46, 41, 45, 48, 51, 49] # 46: open HH
cymbals_pitch = [49, 51] # crash, ride
cymbals_pitch = [] # crash, ride


#mapping midi values into notes
pitch_to_midipitch = {36:Constants.C_3,  # for logic 'SoCal' drum mapping
                        38:Constants.D_3, 
                        39:Constants.Eb_3,
                        41:Constants.F_3,
                        42:Constants.Gb_3,
                        45:Constants.A_3,
                        46:Constants.Bb_3,
                        48:Constants.C_4,
                        49:Constants.Db_4,
                        51:Constants.Eb_4
                        }
#la singola nota è un elemento composto da pitch (numerico, pitch midi) e tick (modo per tenere il tempo in midi)
class Note:
    def __init__(self, pitch, c_tick):
        self.pitch = pitch
        self.c_tick = c_tick # cumulated_tick of a midi note

    def add_index(self, idx):
        '''index --> 16-th note-based index starts from 0'''
        self.idx = idx

class Note_List():
    def __init__(self):
        ''''''
        self.notes = []
        self.quantised = False
        self.max_idx = None

    def add_note(self, note):
        '''note: instance of Note class'''
        self.notes.append(note)

    def quantise(self, minimum_ppq):
        '''
        e.g. if minimum_ppq=120, quantise by 16-th note.
        
        '''
        if not self.quantised:
            for note in self.notes:
                note.c_tick = ((note.c_tick+minimum_ppq/2)//minimum_ppq)* minimum_ppq # quantise
                #here the index is calculated. The index is an absolute index over the 16th notes.
                #for example an index of value 34, means that my current note appears after 34 chromes
                #it's simply calculated by dividing the cumulated tick of the note by the ticks contained in a 16th note
                note.add_index(note.c_tick/minimum_ppq)
            #NB: THE QUANTIZATION FUNCTION ITERATES OVER ALL THE NOTES. So first i add all the notes, then i iterate and quantize

            #Does this automatically reference to the last item added?
            #YES. The counter note will store the last element of the iteration. So basically here i'm assigning as max index the index of the last added note
            self.max_idx = note.idx

            #Here checks if if my ending is a full musical bar. For example, if my file ends with a single kick, i'll add that note.
            #but that kick will (probably) be at the beginning of the last musical bar. So i have to "pad" until the end.
            #It's like adding a pause on my piece, so i have all complete bars and no trucated ones at the end
            if (self.max_idx + 1) % event_per_bar != 0:
                self.max_idx += event_per_bar - ((self.max_idx + 1) % event_per_bar) # make sure it has a FULL bar at the end.
            self.quantised = True

        return

    def simplify_drums(self):
        ''' use only allowed pitch - and converted not allowed pitch to the similar in a sense of drums!
        '''
        #Here forces conversion into the pitches in drum_conversion
        for note in self.notes:
            if note.pitch in drum_conversion: # ignore those not included in the key
                note.pitch = drum_conversion[note.pitch]
        #https://stackoverflow.com/questions/30670310/what-do-brackets-in-a-for-loop-in-python-mean
        #The following one is a list comprehension. Basically generates a new list from an existing one using a given condition on the elements
        self.notes = [note for note in self.notes if note.pitch in allowed_pitch]	

        return

    def return_as_text(self):
        ''''''
        length = int(self.max_idx + 1) # of events in the track.
        #print(type(length))
        event_track = []
        #Thw following cycle create a 9 by N matrix. I append N times a vector of nine zeros.
        #This means that i create N notes, and then i initialize them with all zeros (9 zeros, since a note is represented by a 9 element binary number)

        for note_idx in range(length):  #sostituire xrange con range in Python3
            event_track.append(['0']*len(allowed_pitch))

        num_bars = length/event_per_bar# + ceil(len(event_texts_temp) % _event_per_bar)

        for note in self.notes:
            pitch_here = note.pitch
            #The following line returns the index of the passed pitch. Basically given an input generic pitch
            #it returns the associated pitch in my vocabolary (computes the actual mapping from the whole
            #vocabolary of notes into my reduced one)
            note_add_pitch_index = allowed_pitch.index(pitch_here) # 0-8
            #print(type(note.idx))  
            #print(type(note_add_pitch_index))
            event_track[int(note.idx)][note_add_pitch_index] = '1'
            # print note.idx, note.c_tick, note_add_pitch_index, ''.join(event_track[note.idx])
            # pdb.set_trace()

        event_text_temp = ['0b'+''.join(e) for e in event_track] # encoding to binary

        event_text = []
        # event_text.append('SONG_BEGIN')
        # event_text.append('BAR')
        print(num_bars)
        print(type(num_bars))        
        for bar_idx in range(int(num_bars)):
            event_from = bar_idx * event_per_bar
            event_to = event_from + event_per_bar
            event_text = event_text + event_text_temp[event_from:event_to]
            event_text.append('BAR')

        # event_text.append('SONG_END')

        return ' '.join(event_text)


## Creating Note_List
As simple as that, i'm taking my starting list of couples ```(pitch,time)``` into ```Note```  objects. 

Once I'm done with the generation of the *Note_list* object, I call its utility methods in order to quantise, remove sporious notes (```simplifi_drums()```) and then I convert it in a *.txt* file.




In [ ]:
#Creating note list and appending note objects passing Pitch and Ticks values
note_list = Note_List()
for note in notes:
    pitch = note[0]
    tick = note[1]
    idx = int(tick / min_ppq)
    new_note = Note(pitch,tick)
    new_note.add_index(idx)
    note_list.add_note(new_note)


In [ ]:
for note in note_list.notes:
    print(note.idx)

In [ ]:

print('printing ticks before quantization')
for note in note_list.notes:
    print(note.c_tick)
    
note_list.quantise(min_ppq)

print('printing ticks after quantization')
for note in note_list.notes:
    print(note.c_tick)

In [ ]:
note_list.simplify_drums()

In [ ]:
txt = note_list.return_as_text()

In [ ]:
print(txt)

In [ ]:
text_file = open("datasetartificial.txt", "w")
text_file.write(txt)
text_file.close()

## Testing Txt to MIDI conversion in the following cells

The following cell receives a single line from the txt file until the first 'BAR' element. ```encoded_drums``` is an array where each entry is a note in .txt format, so something like ```0xb011010100```.

```allowed_pitch``` has the following structure ```allowed_pitch = [36, 38, 42, 46, 41, 45, 48, 51, 49]``` where each entry is corresponds to a note in MIDI number (kick, snare ecc..., you can fin the exact notation inside the ```Note_list``` class).

So basically the following function iterates over the single txt note, and for each one of them, iterates over the single binary number and checks if it's equal to one. 
If so, it assigns the equivalent note value taken from ```allowed_pitch```. Basically is doing a 1on1 mapping between the single digit of the binary txt note and the single note taken from ```allowed_pitch```.

In [ ]:
#Function that converts txt to notes. The note is represented as a number (in the MIDI scale)

#in encoded drums ho una riga intera dal file (quindi i vari 0xb00101110) 
def text_to_notes(encoded_drums, note_list=None):
    ''' 
    0b0000000000 0b10000000 ...  -> corresponding note. 
    '''
    if note_list == None:
        note_list = Note_List()
#https://www.programiz.com/python-programming/methods/built-in/enumerate enumerate mi ritorna coppie di (indice,valore) 
    for word_idx, word in enumerate(encoded_drums):
        c_tick_here = word_idx*min_ppq 

        for pitch_idx, pitch in enumerate(allowed_pitch):

            if word[pitch_idx+2] == '1':
                new_note = Note(pitch, c_tick_here)
                note_list.add_note(new_note)
    return note_list

The following function uses ```text_to_notes``` in order to fully convert .txt into MIDI. 

It receives as input the file name

In [ ]:
import os

def conv_text_to_midi(filename):
    if os.path.exists(filename[:-4]+'.mid'):
        return
    f = open(filename, 'r')
    #These multiple readlines are actually useless. Need to check the output of the NN, but right now they're useless.
    #One single readline is enough
    #f.readline() # title
    #f.readline() # seed sentence
    #legge una riga intera dal file
    sentence = f.readline()
    #splitta gli elementi letti a ogni spazio.
    encoded_drums = sentence.split(' ')
    print('printing encoded drums')
    print(encoded_drums)
    #find the first BAR

    first_bar_idx = encoded_drums.index('BAR') 

    #encoded_drums = encoded_drums[first_bar_idx:]
    try:
        encoded_drums = [ele for ele in encoded_drums if ele not in ['BAR', 'SONG_BEGIN', 'SONG_END', '']]
    except:
        pdb.set_trace()

    # prepare output
    note_list = Note_List()
    pattern = Containers.Pattern(fmt=0) #Don't know why there's an assertion in the code for fmt=0 if Pattern.len < 1
    track = Containers.Track()
    #??
    PPQ = 480
    min_ppq = PPQ / (event_per_bar/4)
    track.resolution = PPQ # ???? too slow. why??
    pattern.resolution = PPQ
    # track.resolution = 192
    pattern.append(track)

    velocity = 84
    duration = min_ppq*9/10  # it is easier to set new ticks if duration is shorter than _min_ppq

    note_list = text_to_notes(encoded_drums, note_list=note_list)

    max_c_tick = 0 
    not_yet_offed = [] # set of midi.pitch object
    print('entering for note_idx cycle')
    #In this cycle im adding all the notes except the very last one
    for note_idx, note in enumerate(note_list.notes[:-1]):
        # add onset
        tick_here = note.c_tick - max_c_tick #extracting relative tick
        pitch_here = pitch_to_midipitch[note.pitch]
        # if pitch_here in cymbals_pitch: # "Lazy-off" for cymbals 
        # 	off = midi.NoteOffEvent(tick=0, pitch=pitch_here)
        # 	track.append(off)

        on = Events.NoteOnEvent(tick=tick_here, velocity=velocity, pitch=pitch_here)
        track.append(on)
        max_c_tick = max(max_c_tick, note.c_tick)
        # add offset for something not cymbal

        # if note_list.notes[note_idx+1].c_tick == note.c_tick:
        # 	if pitch_here not in cymbals_pitch:
        # 	# 	not_yet_offed.append(pitch_here)

        # else:
        # check out some note that not off-ed.
        
        #Never enters this cycle. It's useless since we're not adding a NoteOff event anymore. We don't need that
        for off_idx, waiting_pitch in enumerate(not_yet_offed):
            print(off_idx)
            if off_idx == 0:
                off = Events.NoteOffEvent(tick=duration, pitch=waiting_pitch)
                max_c_tick = max_c_tick + duration
            else:
                print('appending end note')
                off = Events.NoteOffEvent(tick=0, pitch=waiting_pitch)
            track.append(off)
            not_yet_offed = [] # set of midi.pitch object 

    # finalise
    if note_list.notes == []:
        print ('No notes in %s' % filename)
        return
        pdb.set_trace()
    #here i'm going to add the last note and close the track with the EndEvent
    note = note_list.notes[-1]
    tick_here = note.c_tick - max_c_tick
    pitch_here = pitch_to_midipitch[note.pitch]
    on = Events.NoteOnEvent(tick=tick_here, velocity=velocity, pitch=pitch_here)
    off = Events.NoteOffEvent(tick=duration, pitch=pitch_here)

    for off_idx, waiting_pitch in enumerate(not_yet_offed):
        off = Events.NoteOffEvent(tick=0, pitch=waiting_pitch)

    # end of track event
    eot = Events.EndOfTrackEvent(tick=1)
    track.append(eot)
    # print pattern
    #print(pattern)
    FileIO.write_midifile(filename[:-4]+'.mid', pattern)


In [ ]:
conv_text_to_midi("semilearnt bar pattern. model 1024.txt")


In [ ]:
text_file = open('metallicaoutput2.txt', "r")
print(text_file.read())